In [2]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
# from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier

from keras import regularizers


import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

In [3]:
# personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SparseAutoencoder'] = {
    'regularizer':['l1', 'l2', 'l1_l2'],
    'reg_constant':[10e-4, 10e-5, 10e-6],
    'encoding_dim': [10, 50, 100, 500],
    'activation': ['relu'],
    'optimizer': ['adadelta'],
    'loss':['binary_crossentropy'],
    'epochs':[10],
    'batch_size':[10, 15]
}

In [4]:
#new data
new_data = pd.read_csv("processed_aml_data.csv")
X = new_data.iloc[:, 1:5051]
y = new_data.iloc[:, 5051]

In [5]:
#convert X and y to numpy
X = X.values
y = y.values

In [6]:
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.20, random_state = 42)
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [7]:
x_train.shape

(42, 5050)

In [8]:
y_train.shape

(42,)

In [9]:
from tpot.builtins import SparseAutoencoder

In [10]:
sae = SparseAutoencoder(encoding_dim=40,
                        regularizer = 'l1',
                        reg_constant = '10e-5',
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=10, batch_size=10)

In [11]:
sae.fit(x_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


SparseAutoencoder(activation='relu', batch_size=10, encoding_dim=40,
         epochs=10, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42, reg_constant='10e-5', regularizer='l1')

In [12]:
tpot = TPOTClassifier(generations=10, config_dict=personal_config,
                        population_size=10, verbosity=3,
                        template = 'SparseAutoencoder-RandomForestClassifier')

In [ ]:
#start = time.time()
tpot.fit(x_train, y_train)
#end = time.time()
#print(end - start)

31 operators have been imported by TPOT.


Generation 1 - Current Pareto front scores:
-2	0.6551587301587302	RandomForestClassifier(SparseAutoencoder(input_matrix, SparseAutoencoder__activation=relu, SparseAutoencoder__batch_size=15, SparseAutoencoder__encoding_dim=100, SparseAutoencoder__epochs=10, SparseAutoencoder__loss=binary_crossentropy, SparseAutoencoder__optimizer=adadelta, SparseAutoencoder__reg_constant=1e-05, SparseAutoencoder__regularizer=l1), RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_features=1.0, RandomForestClassifier__min_samples_leaf=11, RandomForestClassifier__min_samples_split=20, RandomForestClassifier__n_estimators=100)

Generation 2 - Current Pareto front scores:
-2	0.6551587301587302	RandomForestClassifier(SparseAutoencoder(input_matrix, SparseAutoencoder__activation=relu, SparseAutoencoder__batch_size=15, SparseAutoencoder__encoding_dim=100, SparseAutoencoder__epochs=10, SparseAutoencoder__loss=binary_crossentropy, SparseAutoencoder__o

In [ ]:
tpot.fitted_pipeline_

In [ ]:
tpot.score(x_train, y_train)

In [ ]:
tpot.score(x_val, y_val)